81% success of guessing -5 bandit is the most optimal one.

In [1]:
import tensorflow as tf
import numpy as np
import tqdm

/Users/derrkater/anaconda/envs/gym_openai/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
bandits = [0.2, 0, -0.2, -5.]
num_bandits = len(bandits)
def use_bandit(bandit):
    result = np.random.randn(1)  # 1-D np.array with a pseudo random number normal distributed around 0.
    if result > bandit:
        return 1
    else:
        return -1

In [3]:
tf.reset_default_graph()

lr = 0.001

# Forward
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

# Training
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights, action_holder, [1])
loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
update = optimizer.minimize(loss)

In [4]:
total_episodes = 1000
e = 0.1  # Chance of random action
verbose = False

init = tf.initialize_all_variables()

trials_n = 200
trials = []
for t in tqdm.tqdm(range(trials_n)):
    total_reward = np.zeros(num_bandits)
    with tf.Session() as sess:
        sess.run(init)
        for i in range(total_episodes):

            if np.random.rand(1) < e:
                action = np.random.randint(num_bandits)
            else:
                action = sess.run(chosen_action)

            reward = use_bandit(bandits[action])

            feed_dict = {reward_holder: [reward], action_holder: [action]}
            _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict=feed_dict)
            total_reward[action] += reward
            if verbose and not i % 50:
                print("Running reward for the {} bandits: {}.".format(num_bandits, total_reward))
            
    if verbose:
        print("The agent think bandit {} is the most promising...".format(np.argmax(ww)))
        
    trials.append(np.argmax(ww) == 3)
    
sum(trials)/trials_n

Instructions for updating:
Use `tf.global_variables_initializer` instead.


100%|██████████| 200/200 [02:36<00:00,  1.28it/s]


0.81000000000000005